# ベイズ最適化（Bayesian Optimization）
目的関数の評価結果から次の候補を効率的に選ぶ方法  
過去の評価結果を用いて、探索空間内でより有望な領域を重点的に調べる  
探索回数が少なくても効率的に最適パラメータに近づける


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# シード固定
torch.manual_seed(42)
np.random.seed(42)

# サンプルデータ生成：y = 2*x + 1 にノイズを加えた回帰データ
X = np.linspace(-1, 1, 100).reshape(-1, 1)
y = 2 * X + 1 + np.random.normal(0, 0.2, X.shape)

# NumPy → PyTorch tensor
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# シンプルなMLPモデル（1層の隠れ層）
class SimpleMLP(nn.Module):
    def __init__(self, hidden_size):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(1, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# モデル訓練関数：指定したエポック数だけ訓練し、最終MSEを返す
def train_model(hidden_size, lr, epochs=100):
    model = SimpleMLP(hidden_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_tensor)
        loss = criterion(output, y_tensor)
        loss.backward()
        optimizer.step()
    return loss.item()


In [5]:
import optuna

def objective(trial):
    hidden = trial.suggest_int('hidden_size', 4, 32)
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-1)
    loss = train_model(hidden, lr, epochs=100)
    return loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

print("Best parameters (Optuna):", study.best_params, "with Loss:", study.best_value)


[I 2025-02-28 09:03:50,978] A new study created in memory with name: no-name-faa1635b-4f47-4e90-8447-71a108d58179
/var/folders/cd/64q4_xtd5ybfk0cy_yn_56hm0000gn/T/ipykernel_23704/2767309836.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-4, 1e-1)
[I 2025-02-28 09:03:51,539] Trial 0 finished with value: 1.412248969078064 and parameters: {'hidden_size': 6, 'lr': 0.0007178087681879401}. Best is trial 0 with value: 1.412248969078064.
/var/folders/cd/64q4_xtd5ybfk0cy_yn_56hm0000gn/T/ipykernel_23704/2767309836.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-4, 1e-1)
[I 2025-02-28 09:03:51,55

Best parameters (Optuna): {'hidden_size': 32, 'lr': 0.09931864632299335} with Loss: 0.0297447107732296
